# Overview

> You may wish to make a copy of this notebook to avoid overwriting the original.

This drawdown integration connects solutions that impact or are impacted by waste production.  _More about this here_

This notebook is organized into three sections:
* **[Inputs](#Inputs)** takes a look at the inputs that are used to run the integration.  There are defaults for all values, but it is also possible to override those defaults to explore different behaviors.  This section describes the inputs and how they can be set.
* **[Integration Steps](#Integration-Steps)** walks through the steps of the integration process individually.  There is also a single function that will run the entire integraton all at once.
* **[Other Calculations](#Other-Calculations)** shows additional calculations that give insight into the waste model.

In [117]:
import sys
sys.path.append('../')   # If you move this notebook to another location, change this path to point to the root directory of the solutions project
import pandas as pd
from pathlib import Path
from integrations import waste_integration as wi

wi.load_default_values()

# This is an optional step to make floating point numbers display more compactly.
# If you prefer other options, you can change or delete this without harm.
pd.set_option('display.float_format', lambda x: f"{x:03.5f}".rstrip('0'))

# Inputs
<a href='inputs'></a>

We recommend that you look at the contents of file `waste_integration.py`.  The inputs to the integration are in the top section of the file, and each input has a description explaining its content, and usually also it's origin.

All the inputs have default values, most of which are set by the function `load_default_values()`.
Usually `load_default_values()` will be the first function you run (you can see it in the first cell above).
If you want to customize any of the input values, you can simply set the value before running the integration steps.  You can set the value before or after calling `load_default_values()`.

For example, if you wanted to customize which scenarios should be used for the bioplastics adoption, you could do something like the following:
```
    wi.bioplastics_scenario_names = ['custom_pds1_scenario','custom_pds2_scenario','custom_pds3_scenario']
```

As introduction, here are a few of the inputs that are available for inspection or customization:

In [120]:
# The Waste TAM.
# The head() function shows the first n rows of any dataframe

wi.waste_tam.head(10)

,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America,China,India,EU,USA
Year,,,,,,,,,,
2014,1773.33333,531.34132,176.93333,747.38516,255.3121,223.56416,192.52963,142.35164,240.31441,234.4166
2015,1869.53317,549.34356,170.22942,777.82672,269.12705,232.5613,201.29018,151.13914,238.83264,230.78468
2016,1932.45927,557.47683,169.03612,800.24798,281.20763,239.27083,212.63677,161.78695,238.68416,231.50876
2017,1993.62542,565.18782,167.98,822.14045,293.21041,245.835,223.73803,172.33243,238.7006,232.60215
2018,2053.10074,572.49159,167.05529,843.51673,305.1465,252.25785,234.60023,182.77851,238.87585,234.05683
2019,2110.95434,579.40319,166.25622,864.3894,317.02703,258.54342,245.22967,193.12813,239.20377,235.8648
2020,2167.25532,585.9377,165.57704,884.77107,328.86311,264.69575,255.63265,203.38423,239.67825,238.01805
2021,2222.0728,592.11015,165.01197,904.67433,340.66586,270.71887,265.81545,213.54974,240.29316,240.50859
2022,2275.47588,597.93562,164.55525,924.11176,352.44641,276.61682,275.78437,223.6276,241.04238,243.32839


In [11]:
# The amount of food waste, in PDS1, PDS2 and PDS3 scenarios

wi.food_waste.head(10)

,PDS1,PDS2,PDS3
Year,,,
2014,826.15184,826.15184,826.15184
2015,837.16193,837.16193,837.16193
2016,848.09984,848.09984,848.09984
2017,858.96102,858.96102,858.96102
2018,869.73361,869.73361,869.73361
2019,880.41176,880.41176,880.41176
2020,890.99123,890.99123,890.99123
2021,901.45993,901.45993,901.45993
2022,911.81252,911.81252,911.81252


This is the complete list of input variables that are used in the integration:

`waste_tam`, `compostable_proportion`, `recyclable_proportion`, `regional_organic_fraction`, `regional_recyclable_fraction`, `plastics_worldwide`, `proportion_plastics_in_waste`, `bioplastics_adoption`, `bioplastics_scenario_names`, `food_waste`, `food_waste_reduction`


# Integration Steps

# Other Calculations

<hr/>

### Developer Notes

> The remainder of this notebook is notes of the developer, and can be ignored.

Analysis of formula references:
```
    (PDS1, PDS2, PDS3) <--> (WTE Tonnes & Composition, Paper, Compost, Recycling)
    (PDS1, PDS2, PDS3) <-- Food System Summary
    PDS1 <--> Bioplastics
    (WTE Tonnes & Composition <--> WTE and LM Analysis <--> Landfill Methane)  <-- (PDS1, PDS2, PDS3)
    Refrigerant Management  <-->  None
```
That is, PDS1,2,3 are both input and output for the topic tabs, and the Waste & Landfill Methand tabs are a tightly bound cluster.  Refrigerant Management is isolated, as are all the background sheets, and the MSW TAM Update.

For some reason Bioplastics are tied only with PDS1?

The instruction document is quite detailed and helpful.  We'll mostly follow along.

In [30]:
import openpyxl
from model import sma
from tools import sma_xls_extract
import warnings
from solution import factory

warnings.simplefilter("ignore")  # silence openpyxl warnings

In [116]:
import importlib
importlib.reload(wi)
wi.load_default_values()

In [8]:
excel_file = Path(r'C:\Working\Integrations\Materials and Waste Integration Analysis_June2020.xlsm')
wb = openpyxl.load_workbook(excel_file, data_only=True, keep_links=False)

### Waste TAM

In [9]:
# Step one:  Save the "Waste TAM" to a data file.
# (Note: the instructions say to update the Waste TAM from another integration step; for now 
# are just going to assume this has been done already)
wastetam = sma_xls_extract.extract_sma(wb, 'Total MSW TAM Data_Updated2020', year_col='B', 
                                       start_row=41, use_functional_unit=True)

In [10]:
wastetam.region_cases

{'World': {'Base Cases': ['S1', 'S2'], 'Conservative Cases': ['S3']},
 'OECD90': {'Base Cases': ['S1', 'S2'], 'Ambitious Cases': ['S4']},
 'Eastern Europe': {'Base Cases': ['S1', 'S2'], 'Ambitious Cases': ['S4']},
 'Asia (Sans Japan)': {'Base Cases': ['S1', 'S2'], 'Ambitious Cases': ['S4']},
 'Middle East and Africa': {'Base Cases': ['S1', 'S2', 'S5'],
  'Ambitious Cases': ['S4']},
 'Latin America': {'Base Cases': ['S1', 'S2'], 'Ambitious Cases': ['S4']},
 'China': {'Base Cases': ['S6'], 'Ambitious Cases': ['S7']},
 'India': {'Base Cases': ['S6', 'S8'], 'Ambitious Cases': ['S7', 'S9']},
 'EU': {'Base Cases': ['S6', 'S10'], 'Ambitious Cases': ['S7']},
 'USA': {'Base Cases': ['S6', 'S11'], 'Ambitious Cases': ['S7']}}

In [11]:
wastetam.write(r'data\msw','waste_tam')

In [7]:
wb.close()

In [3]:
wt2 = sma.SMA.read(r'data\msw',"waste_tam")

In [6]:
wt2.sources['S1'].data.head()

,Year,World,OECD90,Eastern Europe,Asia (Sans Japan),Middle East and Africa,Latin America
0,2012,1536.666667,520.994064,174.333333,714.887215,227.431963,215.467580
1,2013,1655.000000,526.167694,175.633333,731.136187,241.372032,219.515868
2,2014,1773.333333,531.341324,176.933333,747.385160,255.312100,223.564155
3,2015,1891.666667,536.514954,178.233333,763.634132,269.252169,227.612443
4,2016,2010.000000,535.700000,178.200000,769.100000,302.400000,230.800000


### Step Two: Subject Sources as SMAs

In [3]:
# This is really just manual work, copying the current values of the tables, e.g. Compost!I3:J52 into SMA data format.
# I suppose I could write code to do this, but it doesn't seem worth it.
# Key note:  I change the number format to six digits precision before copying.
# Other Note:  The instructions to the integration say to get these data from the corresponding model (e.g. composting), 
# but this is not a part of those models we are saving or representing currently.
# Example result:
cf = sma.SMA.read( Path("data/msw"), "organic_compost_sma" )
cf.summary(case='S1').head()

,World
Year,
2012,44.831749
2013,45.099481
2014,45.346352
2015,45.573771
2016,46.000000


In [ ]:
# And that is more or less it.  Everything goes into SMAs or plain csvs, or directly into the code.

In [112]:
get_feedstock_limited_solutions()

{'composting':       PDS1  PDS2      PDS3
 Year                      
 2014    0.    0.        0.
 2015    0.    0.        0.
 2016    0.    0.        0.
 2017    0.    0.        0.
 2018    0.    0.        0.
 2019    0.    0.        0.
 2020    0.    0.        0.
 2021    0.    0.        0.
 2022    0.    0.        0.
 2023    0.    0.        0.
 2024    0.    0.        0.
 2025    0.    0.        0.
 2026    0.    0.        0.
 2027    0.    0.        0.
 2028    0.    0.        0.
 2029    0.    0.        0.
 2030    0.    0.        0.
 2031    0.    0.        0.
 2032    0.    0.        0.
 2033    0.    0.        0.
 2034    0.    0.        0.
 2035    0.    0.  34.04309
 2036    0.    0.  85.56514
 2037    0.    0. 124.56927
 2038    0.    0. 163.75078
 2039    0.    0. 203.04917
 2040    0.    0. 239.32814
 2041    0.    0.   274.005
 2042    0.    0. 306.87244
 2043    0.    0. 338.48314
 2044    0.    0. 369.71564
 2045    0.    0. 400.55121
 2046    0.    0. 420.05784
 2047 

In [104]:
old_compost_adoption = wi.compost_adoption.copy()

In [105]:
old_feedstock = wi.get_updated_organic_waste().copy()

In [114]:
wi.update_solution_adoptions(1)